In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/data/SSH/SSH_2005-2015_month.nc')

In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))
ssh_lat = data_ssh['lat'][:].data
ssh_lon = data_ssh['lon'][:].data
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

132


In [7]:
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/data/SSS/SSS_2005-2020_month.nc')

In [9]:
time_sss = data_sss['time'][:132]
print(len(time_sss))
sss_lat = data_sss['lat'][:].data
sss_lon = data_sss['lon'][:].data
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)
print(sss.shape)

132


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/data/SSW/SSW_2005-2018_month.nc')

In [14]:
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))
lat_ssw = data_ssw['lat'][:].data
lon_ssw = data_ssw['lon'][:].data

132


In [16]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## 3DT

In [20]:
data_3dt_1 = nc.Dataset(r'D:/data/3DTS/3DT_2005-2009_month.nc')
data_3dt_2 = nc.Dataset(r'D:/data/3DTS/3DT_2010-2015_month.nc')

In [21]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data
print(sst_1.shape)
print(sst_2.shape)

###  5 - 2000m 温度数据

In [23]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [24]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
depths_use = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

In [26]:
temp_depth_use = []
selected_layers = []
for i, depth in enumerate(depths):
    if depth in depths_use:
        selected_layers.append(temp_depth[:, i:i+1, :, :])
if selected_layers:
    temp_depth_use = np.concatenate(selected_layers, axis=1)
else:
    temp_depth_use = None

In [27]:
print(temp_depth_use.shape)
temp_depth_use = np.transpose(temp_depth_use, (0,2,3,1))
print(temp_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## SST

In [31]:
data_sst = nc.Dataset('D:/data/SST/SST_2005-2015_month.nc','r')

In [32]:
time_sst = data_sst['time'][:]
print(len(time_sst))
sst_lat = data_sst['lat'][:].data
sst_lon = data_sst['lon'][:].data
sst = data_sst['sst'][:].data
print(ssh.shape)

132


In [34]:
np.nanmax(sst),np.nanmin(sst)

(31.901611, -1.8474731)

## 数据合并

In [35]:
print(ssh.shape,sss.shape,uwnd.shape,vwnd.shape,sst.shape)
cat_data =np.stack((ssh,sss,uwnd,vwnd,sst),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [37]:
cat_data_ss_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)
cat_data_ss_depth.shape

(132, 280, 600, 38)

## 将两个温度异常区域的数据设置为nan

In [38]:
test = temp_depth_use[:,:,:,27:28]
test[test < 0.4] = np.nan
test[test > 10] = np.nan
cat_data_ss_depth[:,:,:,32:33] = test

## 将填充值赋值为NAN，且一个点中有一个维度为nan，则将这个点的所有维度都设置为nan 


In [41]:
cat_data_ss_depth_reshape  = np.transpose(cat_data_ss_depth,(1,2,0,3))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape ((-1,132*38))
print(cat_data_ss_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [42]:
cat_data_ss_depth_reshape[cat_data_ss_depth_reshape == 32767] = np.nan
nan_mask = np.isnan(cat_data_ss_depth_reshape)
np.sum(nan_mask.any(axis=1) == True)

In [44]:
cat_data_ss_depth_reshape[nan_mask.any(axis=1) == True] = np.nan   
print('cat_data_ss_depth_reshape:',cat_data_ss_depth_reshape.shape)

cat_data_ss_depth_reshape: (168000, 5016)


In [45]:
np.sum(np.isnan(cat_data_ss_depth_reshape[:,4])) 

64728

In [46]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,132,38)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = np.transpose(cat_data_ss_depth_reshape,(1,0,2))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,280,600,38)
print(cat_data_ss_depth_reshape.shape)

(168000, 132, 38)
(132, 168000, 38)


In [48]:
np.sum(np.isnan(cat_data_ss_depth_reshape[1:2,:,:,2:3]))

64728

## 将数据分成小块

In [49]:
cat_data_ss = cat_data_ss_depth_reshape[:,:,:,:5] 
cat_data_depth = cat_data_ss_depth_reshape[:,:,:,5:] 
print(cat_data_ss.shape,cat_data_depth.shape)

(132, 280, 600, 5) (132, 280, 600, 33)


In [50]:
np.sum(np.isnan(cat_data_depth[1:2,:,:,5:6]))

64728

In [51]:
import numpy as np
import pandas as pd

time_size, lat_size, lon_size, channel = cat_data_ss.shape 
          
box_size = 9 
step_size = 1 

out_lat_size = (lat_size - box_size) // step_size  + 1 
out_lon_size = (lon_size - box_size) // step_size  + 1 

print('time_size:',time_size,'lat_size:',lat_size,'lon_size:',lon_size) 
print(out_lat_size,out_lon_size)  #(272 592)

out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) 
out_data_y = np.zeros((time_size, out_lat_size , out_lon_size, 33))

for t in range(time_size):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1, step_size):
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            box_data = cat_data_ss[t, lat_min:lat_max, lon_min:lon_max,:]
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []
            sample_idx += 1

print('out_data的形状',out_data.shape)
    
#==============================================================================#   

for t in range(time_size):
    sample_idx = 0
    for lat in range(4, lat_size-4, step_size):
        for lon in range(4, lon_size-4, step_size):
            box_data_y = cat_data_depth[t, lat:lat+1, lon:lon+1,:]
            out_data_y[t,lat-4,lon-4, :] = box_data_y
            box_data_y = []
            sample_idx += 1

print('out_data_y的形状',out_data_y.shape)

time_size: 132 lat_size: 280 lon_size: 600
272 592
out_data的形状 (132, 272, 592, 9, 9, 5)
out_data_y的形状 (132, 272, 592, 33)


## 删除缺失值多于一半的数据

### 获取NAN_mask 

In [53]:
out_data_reshape = out_data.reshape((132,-1,9,9,5))
print(out_data_reshape.shape) 
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

(132, 161024, 9, 9, 5)
(132, 161024, 9, 9, 5)


In [54]:
np.count_nonzero(nan_mask_out_data[7:8,:,:,:,2:3])

4880718

In [55]:
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,0] 
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) 
np.count_nonzero(nan_mask_out_data_one) 
nan_mask_out_data_one.shape 
nan_mask_out_data_one

### 为有效值大于一半的滑块标注True

In [57]:
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40): 
        nan_mask.append(True)
    else:
        nan_mask.append(False)

In [58]:
np.count_nonzero(nan_mask) 

101223

In [59]:
out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,33)
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (161024, 132, 9, 9, 5)
out_data_y_reshape: (272, 592, 132, 33)
out_data_y_reshape: (161024, 132, 33)


## 开始删除无效数据

In [60]:
out_data_reshape = out_data_reshape[nan_mask]
out_data_y_reshape = out_data_y_reshape[nan_mask]
print(out_data_reshape.shape,out_data_y_reshape.shape)

(101223, 132, 9, 9, 5) (101223, 132, 33)


## 使用0来填充nan 

In [61]:
out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)  
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)

# 验证集和测试集的划分

## 对数据进行reshape,以进行数据集的划分

In [62]:
out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (132, 101223, 9, 9, 5)
out_data_y_reshape: (132, 101223, 33)


In [63]:
test_num = 120
x_train = out_data_reshape[:test_num]
y_train = out_data_y_reshape[:test_num]

x_test = out_data_reshape[test_num:]
y_test = out_data_y_reshape[test_num:]

In [63]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape 

((120, 101392, 9, 9, 5),
 (120, 101392, 33),
 (12, 101392, 9, 9, 5),
 (12, 101392, 33))

In [64]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))

y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))

## 划分训练数据和目标数据

In [65]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1214676, 9, 9, 5)
y_test.shape (1214676, 33)
x_train (12146760, 9, 9, 5)
y_train (12146760, 33)


In [66]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)

In [67]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8502732, 9, 9, 5)
x_test (1214676, 9, 9, 5)
x_val (3644028, 9, 9, 5)
y_train (8502732, 33)
y_test (1214676, 33)
y_val (3644028, 33)


# 特征归一化

In [69]:
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [70]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[3])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[3]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[3]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[1])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[1])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[1])) 

In [71]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))
x_val = np.reshape(x_val,(-1,9,9,5))
y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))
y_val = np.reshape(y_val,(-1,33))

In [72]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8502732, 9, 9, 5)
x_test (1214676, 9, 9, 5)
x_val (3644028, 9, 9, 5)
y_train (8502732, 33)
y_test (1214676, 33)
y_val (3644028, 33)


# 定义网络结构

In [73]:
def ChannelAttention(inputs,in_planes, ratio=2):
    
    avg_out= layers.GlobalAveragePooling2D()
    max_out= layers.GlobalMaxPooling2D()

    fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            activation=tf.nn.relu,
                            use_bias=True, bias_initializer='zeros')
    fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            use_bias=True, bias_initializer='zeros')

    
    avg_out = avg_out(inputs)
    max_out = max_out(inputs)
    out = tf.stack([avg_out, max_out], axis=1)
    out = fc2(fc1(out))
    out = tf.reduce_sum(out, axis=1)
    out = tf.nn.sigmoid(out)
    out = layers.Reshape((1, 1, out.shape[1]))(out)

    return  out*inputs

In [74]:
def regularized_padded_conv(*args, **kwargs):
    return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(5e-4))

def SpatialAttention(inputs,kernel_size=5):
    conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
    avg_out = tf.reduce_mean(inputs, axis=3)
    max_out = tf.reduce_max(inputs, axis=3)
    out = tf.stack([avg_out, max_out], axis=-1)        
    out = conv1(out)
    return out*inputs

In [75]:
def cbam_block(inputs, in_planes,ratio=2):
    cbam_feature = ChannelAttention(inputs, in_planes,ratio)
    cbam_feature = SpatialAttention(cbam_feature)
    return cbam_feature

In [76]:
def create_model():
    model_input= Input(shape=x_train.shape[1:])
    output_size = 1
    x = cbam_block(model_input,5)
    x = Conv2D(filters=16, kernel_size=(3, 3),strides=1)(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,16)
    x = Conv2D(filters=32, kernel_size=(3, 3),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,32)
    x = Flatten()(x)

    x = Dense(units=16)(x)
    x = Activation('relu')(x)

    out = Dense(units=output_size)(x)
    
    model = Model(inputs=model_input, outputs=out)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 

In [ ]:
model = create_model()
model.summary()

## 训练

In [78]:
def train_model_CNN(x_train,y_train,x_val,y_val,name):
    model = create_model()
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [ ]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33):
    train_model_CNN(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],
                'D:/CNN/temp/CNN_temp_'+str(depth[i])+'m.h5')

## 模型评估

In [95]:
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)

def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()

def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)

    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" % r2)  

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

### 结果评估

In [ ]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/CNN/temp/CNN_temp_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m温度的评价')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

### 反归一化结果评价

In [ ]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)
y_test = y_test.reshape(-1,33)
print(y_test.shape)

In [ ]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)
result_unscaled.shape,y_test.shape

In [ ]:
for i in range(0,33):
    print('预测第'+str(depth[i])+'m温度的评价')
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print('R² score:',r2)

# 预测结果可视化

## 为预测值和真实值填充nan

In [115]:
testPred_with_nan = np.full((161024,12*33), np.nan)  
print(testPred_with_nan.shape)
y_test_with_nan = np.full((161024,12*33), np.nan)  
print(y_test_with_nan.shape)

(161024, 396)
(161024, 396)


In [116]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ==================================================================== #

y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)

(12, 101392, 33)
(101392, 12, 33)
(101392, 396)
(12, 101392, 33)
(101392, 12, 33)
(101392, 396)


In [117]:
y_test.shape, y_test_reshape.shape

(1216704, 33)

In [118]:
num = 0
for i,nanmask in enumerate(nan_mask):  
    if (nanmask ==True): 
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1    
print(num)

101392


In [119]:
testPred_with_nan = testPred_with_nan.reshape(272, 592,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))

y_test_with_nan = y_test_with_nan.reshape(272, 592,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 272, 592) (12, 33, 272, 592)


# 结果可视化

In [120]:
lat = []
lon = []
time = []
lon.append(data_ssh['lon'][4:-4].data)
lat.append(data_ssh['lat'][4:-4].data)

In [122]:
len(lat),len(lat[0]),len(lon),len(lon[0])

(1, 272, 1, 592)

In [123]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [126]:
new_NC = nc.Dataset("D:/CNN/result_temp_CNN_CBAM_5_2000m.nc",
                    'w', format='NETCDF4')
'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('Observed_ST', 'f',("time","depth","lat","lon"))
new_NC.createVariable('Estimated_ST', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['Observed_ST'][:]=np.array(y_test_with_nan)
new_NC.variables['Estimated_ST'][:]=np.array(testPred_with_nan)

new_NC.close()